<a href="https://colab.research.google.com/github/christopher-c-lee/machine-learning-projects/blob/main/classification-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
#### Basic idea for Decision Trees look at one variable at a time.
#### So no need to scale the data. 
#### Also since 

In [ ]:
titanic_unscaled = pd.read_pickle("Titanic_PreprocessedData.pkl")

ValueError: ignored

In [ ]:
titanic_unscaled.head()

#### Now let us divide this into training and validation dataset  - 70% and 30%

### We are going to use a library called scikit learn (denoted by sklearn) for predictive modeling. It has all the methods we need to build predictive models. 


###  We split the cleaned up data into training and validation sets. (scikit learn calls them training and test) 


In [ ]:
from sklearn.model_selection import train_test_split 
# importing the specific method for randomly splitting the data 
# Note that this a weird method, it gives 4 dataframes as output ( the specific order is important, the specific names are not) ! 
## The input to the method is the the set of predictor columns, The single target column, the split rati0
# The random_state allows you to create repeatable random splitting. So you get the same split everytime you run it.  

In [ ]:
train_predictors, valid_predictors, train_target, valid_target = train_test_split(titanic_unscaled.drop(columns = {"survived"}), 
                                                    titanic_unscaled[["survived"]], test_size=0.3, random_state=1)

NameError: ignored

In [ ]:
train_predictors.shape

In [ ]:
train_predictors.head()

In [ ]:
valid_predictors.shape

In [ ]:
train_target.shape

In [ ]:
valid_target.shape

In [ ]:
valid_target.columns

### Now we are ready to build a decision tree  model  on the training data 
### Remember we always build our model only using the training data 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree_titanic = DecisionTreeClassifier(max_depth = 4, min_samples_split=40,min_samples_leaf = 10) # These are parameters that control the tree we build
tree_titanic.fit(train_predictors,train_target)




### Now that the model is created, we can apply it to the training and  validation data set to see how accurate its predictons are on the training and validation data set.  Each model type has a .predict method to you which you pass the predictor columns. 


In [ ]:
tree_titanic_train_predict = pd.DataFrame(tree_titanic.predict(train_predictors), index = train_target.index, columns = {"Tree_prediction"} )
tree_titanic_train_predict

In [ ]:
tree_titanic_valid_predict = pd.DataFrame(tree_titanic.predict(valid_predictors), index = valid_target.index, columns = {"Tree_prediction"} )
tree_titanic_valid_predict

In [ ]:
## Since we want a dataframe
tree_titanic_valid_predict = pd.DataFrame(tree_titanic.predict(valid_predictors), index = valid_target.index, columns = {"tree_prediction"} )
tree_titanic_valid_predict

In [ ]:
valid_target

In [ ]:
from dmba import classificationSummary # 
from sklearn.metrics import classification_report

In [ ]:
classes = ['Died', 'Survived']

In [ ]:
print("TRAINING CONFUSION MATRIX")
classificationSummary(train_target,tree_titanic_train_predict, class_names=classes) 

print("VALIDATION CONFUSION MATRIX")

classificationSummary(valid_target,tree_titanic_valid_predict, class_names=classes) 

In [ ]:
valid_target.value_counts() ### checking that the numbers add up ! 

In [ ]:
print(classification_report(valid_target,tree_titanic_valid_predict, target_names=classes))

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pylab as plt

In [ ]:
plt.figure(figsize=(18,12))  # set plot size (denoted in inches)
plot_tree(tree_titanic, feature_names=list(train_predictors.columns),filled=True, fontsize=10)
plt.show

#### Class Exercise: Consider the decision tree above. 

#### 1.  We just learned about a mystery passenger from some of the survivors who happened to have her ticket - She was a 21 year old  female, who was a 3rd  class passenger, who embarked at Southamption and who had paid a fare of 18 pounds. 

#### 2. Write down the all prediction rules that results from the above decision tree. 


### Now let us fit Random Forests 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#### Now let us fit a random forest model. It is almost exactly the same syntax as for decision trees. 

In [ ]:
rf_titanic = RandomForestClassifier(n_estimators=20, random_state=1)
rf_titanic.fit(train_predictors,train_target)


In [ ]:
rf_titanic_train_predict = pd.DataFrame(rf_titanic.predict(train_predictors), index = train_target.index, columns = {"rf_prediction"} )
rf_titanic_train_predict

In [ ]:
rf_titanic_valid_predict = pd.DataFrame(rf_titanic.predict(valid_predictors), index = valid_target.index, columns = {"rf_prediction"} )
rf_titanic_valid_predict

#### How good is this random forest model 

In [ ]:
print("TRAINING CONFUSION MATRIX")
classificationSummary(train_target,rf_titanic_train_predict, class_names=classes) 

print("VALIDATION CONFUSION MATRIX")

classificationSummary(valid_target,rf_titanic_valid_predict, class_names=classes) 

### Random Forest Models also assign class propensities which are extremely when predicting one class correctly is more important that the other due to the business context

In [ ]:
rf_titanic.predict_proba(valid_predictors)

In [ ]:
rf_results = pd.DataFrame(rf_titanic.predict_proba(valid_predictors), index = valid_target.index, columns = {"propensity_for _0", "propensity_for_1"} )

In [ ]:
rf_results.head()

In [ ]:
rf_results["rf_prediction"] = rf_titanic.predict(valid_predictors)

In [ ]:
rf_results["Actual Class"] = valid_target

In [ ]:
rf_results

In [ ]:
rf_results.rf_prediction.sum() ## confirming the number of predicted ( not actual survivals) from validation dataset 

In [ ]:
sorted_rf_results = rf_results.sort_values("propensity_for_1")

In [ ]:
sorted_rf_results.reset_index()

In [ ]:
sorted_rf_results.head()

In [ ]:
sorted_rf_results[160:180] ### To see where the prediction changes from 0 to 1

In [ ]:
forest_importances = pd.DataFrame(rf_titanic.feature_importances_, columns = {"Importance"})
forest_importances

In [ ]:
forest_importances["Predictor Names"] =train_predictors.columns

In [ ]:
forest_importances

#### The above numbers give you a quantitative measure of the proportional importance of each predictor